# Geração por Usina por Base Horária

## Code

In [1]:
import panel as pn
import pandas as pd
import hvplot.pandas
from infra_copel import MongoHistoricoOficial
from datetime import datetime, date, timedelta

In [10]:
# Conectar ao MongoDB
def collection_connect(collection_name: str):
    mongo = MongoHistoricoOficial()
    collection = mongo[collection_name]
    return collection

def response(usina_name, initial_date, final_date):
    start_date = datetime(initial_date.year, initial_date.month, initial_date.day)
    end_date = datetime(final_date.year, final_date.month, final_date.day)

    try:
        result = collection.find(
            {'usina': usina_name, 'periodo': {'$gte': start_date, '$lt': end_date}},
            {'_id': 0, 'periodo': 1, 'geracao_mwm': 1}
        )
        result_list = list(result)
        if result_list:
            return create_chart(result_list, usina_name)
        else:
            return "Nenhum resultado encontrado para o período especificado."
        
    except Exception as e:
        return f"Erro ao acessar o banco de dados: {e}"

def create_chart(data: list[dict], usina: str):
    df = pd.DataFrame(data)
    return df.hvplot(x='periodo', y='geracao_mwm', kind='line', title=f'Geração da Usina - {usina}', width=500, height=400, grid=True)

def filter_usina(collection):
    usinas = collection.distinct("usina")
    usina_widget = pn.widgets.Select(options=usinas, name="Selecione usina: ", value="Furnas", styles={"color": "black", "background-color": "grey"})
    return usina_widget

def filter_dates():
    data_pesquisa_inicial = date.today() - timedelta(days=200)
    data_pesquisa_final = date.today() - timedelta(days=7)
    filtro_data_initial = pn.widgets.DatePicker(name="Data Inicial", value=data_pesquisa_inicial, styles={"color": "black", "background-color": "grey"})
    filtro_data_final = pn.widgets.DatePicker(name="Data Final", value=data_pesquisa_final, styles={"color": "black", "background-color": "grey"})
    return filtro_data_initial, filtro_data_final

def dashboard(usina, primeira_data, segunda_data):
    """ panel.widgets.select.Select"""
    return pn.panel(
    pn.Column(
        pn.Column(usina, primeira_data, segunda_data),
        pn.bind(
        response, usina_name= usina, initial_date=primeira_data, final_date=segunda_data)
    )
)

In [11]:
def main():

    # Constants
    COLLECTION_NAME = "ons_geracao_usinas"
    TITLE = pn.pane.HTML("<h1>Geração por Usina por Base Horária</h1>", styles={"color": "orange"})
    INFO1 = pn.panel("Aguarde alguns segundos o carregamento dos dados.", styles={"color": "orange"})
    INFO2 = pn.panel("E observe se a data inicial é maior do que a final.", styles={"color": "black"})

    # Connect to MongoDB
    collection = collection_connect(COLLECTION_NAME)

    # Widgets
    filter_usina_1 = filter_usina(collection=collection)
    filter_usina_2 = filter_usina(collection=collection)
    dash_1_initial_date, dash_1_final_date = filter_dates()
    dash_2_initial_date, dash_2_final_date = filter_dates()

    primeiro_dashboard = dashboard(usina=filter_usina_1,  primeira_data=dash_1_initial_date, segunda_data=dash_1_final_date)
    segundo_dashboard = dashboard(usina=filter_usina_2,  primeira_data=dash_2_initial_date, segunda_data=dash_2_final_date)

    # Display dashboards
    return pn.Column(
    TITLE,
    pn.Row(primeiro_dashboard, segundo_dashboard),
    INFO1,
    INFO2
    )

## Dashboard

In [12]:
main()

[2024-01-05T19:45:50.566-0300] {base.py:73} INFO - Using connection ID 'historico_oficial' for task execution.


Column
    [0] HTML(str, styles={'color': 'orange'})
    [1] Row
        [0] Column
            [0] Column
                [0] Select(name='Selecione usina: ', options=['14 de Julho', ...], styles={'color': 'black', ...}, value='Furnas')
                [1] DatePicker(name='Data Inicial', styles={'color': 'black', ...}, value=datetime.date(2023, ...)
                [2] DatePicker(name='Data Final', styles={'color': 'black', ...}, value=datetime.date(2023, ...)
            [1] ParamFunction(function, _pane=Markdown, defer_load=False)
        [1] Column
            [0] Column
                [0] Select(name='Selecione usina: ', options=['14 de Julho', ...], styles={'color': 'black', ...}, value='Furnas')
                [1] DatePicker(name='Data Inicial', styles={'color': 'black', ...}, value=datetime.date(2023, ...)
                [2] DatePicker(name='Data Final', styles={'color': 'black', ...}, value=datetime.date(2023, ...)
            [1] ParamFunction(function, _pane=Markdown, defer_load=False)
    [2] Markdown(str, styles={'color': 'orange'})
    [3] Markdown(str, styles={'color': 'black'})